In [5]:
import cvxpy as cp
import numpy as np
from datetime import datetime, timedelta

# Constants to eliminate hardcoding
NUM_MOVIES = 8
NUM_TIMESLOTS = 41
NUM_THEATERS = 8

profit = cp.Variable((NUM_MOVIES, NUM_TIMESLOTS), nonneg=True)  # Profit of movie i at time j
tickets = cp.Variable((NUM_MOVIES, NUM_TIMESLOTS), integer=True)  # Number of tickets sold for movie i at time j
showings = cp.Variable((NUM_MOVIES, NUM_TIMESLOTS), integer=True)  # Boolean if movie i is shown at time j
theaters = [cp.Variable((NUM_MOVIES, NUM_TIMESLOTS), boolean=True) for _ in range(NUM_THEATERS)]  # A boolean matrix that shows if theater k is displaying a movie

# Movies to be shown at specific timeslots
movies = ["Gladiator 2", "Anora", "Wicked", "A Real Pain", "Life is Beautiful", "Heretic", "Basquiat", "American Werewolf in London"]

# Movie popularity index, which is derived from current position on charts
# If movie is not currently on charts or was released long ago, use approximate based on peak
mPopularity = np.array([.99, .93, .94, .66, .73, .95, .25, .15]) # Popularity of movie (i)

# Length of movie
movieLength = [10, 10, 11, 6, 8, 7, 7, 6]

# Capacity of each theater, found from the gateway website for ticketing
theaterCapacity = [113, 113, 127, 312, 72, 72, 113, 44]

# The popularity of each timeslot, which was found via google analytics and reviews
tPopularity = np.array([0.07, 0.07, 0.07, 0.07, 0.2, 0.2, 0.2, 0.2, 0.35, 0.35, 0.35, 0.35, 0.45, 0.45, 0.45, 0.45, 0.6, 0.6, 0.6, 0.6, 0.75, 
               0.75, 0.75, 0.75, 0.85, 0.85, 0.85, 0.85, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.8]) # Popularity of timeslot (j)

# Price vector of each timeslot
price = [9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75,
         13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50]


constraints = []

# Each movie can be shown in at most 5 time slots
for i in range(NUM_MOVIES):
    constraints.append(sum(showings[i]) <= 5)
    constraints.append(sum(showings[i]) >= 1)
    for j in range(NUM_TIMESLOTS):
        constraints.append(cp.sum(showings[:, j]) <= 2)
        # Define profit as tickets sold * price
        constraints.append(profit[i, j] == tickets[i, j] * price[j] * mPopularity[i] * tPopularity[j] )

        # Sum of theaters showing a movie cannot exceed total theaters
        constraints.append(sum(theaters[k][i, j] for k in range(NUM_THEATERS)) == showings[i, j])

        # Tickets sold = sum of theater capacities for the movie
        constraints.append(
            tickets[i, j] == sum(theaterCapacity[k] * theaters[k][i, j] for k in range(NUM_THEATERS))
        )

        # A movie cannot start 10 time slots before or after a movie in the same theater
        start = max(0, j)  # Ensure we don't go below index 0
        end = min(NUM_TIMESLOTS, j + movieLength[i] + 1)  # Ensure we don't go beyond index 40 (inclusive)

        # Ensure only one movie is scheduled in the range from j-11 to j+11 for the current theater
        for k in range(NUM_THEATERS):  # Loop over all theaters
            constraints.append(
                cp.sum([theaters[k][i, t] for i in range(NUM_MOVIES) for t in range(start, end)]) <= 1
            )
            
           
obj_func = cp.sum(profit)

problem = cp.Problem(cp.Maximize(obj_func), constraints)

problem.solve(solver=cp.GUROBI, verbose=True)

print("Optimal objective value =", obj_func.value)
print("Profit =")
print(profit.value)
print("Tickets sold =")
print(tickets.value)
print("Showings =")
print(showings.value)
print("Theaters =")
for k in range(8):  # Loop through each theater
    print(f"Theater {k + 1}:")
    temp = theaters[k].value
    # Start time is 1:00 PM, each slot is 15 minutes
    start_time = datetime.strptime("1:00 PM", "%I:%M %p")

    for i in range(8):  # Loop through each movie
        for j in range(41):  # Loop through each time slot
            if temp[i, j].astype(int) == 1:
                # Calculate the actual time for the slot
                current_time = start_time + timedelta(minutes=15 * j)
                # Format the time as a string (e.g., "1:00 PM")
                formatted_time = current_time.strftime("%I:%M %p")
                print(f"{formatted_time}: {movies[i]}")
        
    print(theaters[k].value)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 21 10:52:09 PM: Your problem has 3608 variables, 3952 constraints, and 0 parameters.
(CVXPY) Nov 21 10:52:12 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 21 10:52:12 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 21 10:52:12 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 21 10:52:12 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 21 10:52:15 PM: Compiling problem (target solver=GUROBI